In [1]:
import os, tarfile
import pyBigWig
import pybedtools
import zipfile, gzip
from gtfparse import read_gtf
import pandas as pd
pd.set_option('display.max_columns', None)
from Bio.Seq import Seq

In [2]:
reference_genome_path = "/mnt/data05/shared/pdutta_data/Human_Genome_Data/UCSC/genome_by_sequence/"
data_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/UCSC/gencode_annotation/"
file_name="gencodeV38.bb"
gencode_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/gencode_annotation/"
gtf_annotation_filename= "gencode.v38.annotation.gtf"
gff_annotation_filename= "gencode.v38.annotation.gff3"
file_path= data_path+file_name
gencode_gtf_file_path= gencode_path+ gtf_annotation_filename
gencode_gff_file_path= gencode_path+ gff_annotation_filename
#reference_genome_file_path = reference_genome_path + reference_genome_name
data_save_path= "/mnt/data05/shared/pdutta_data/Human_Genome_Data/region_wise_sequence/"

In [3]:
chr_files= os.listdir(reference_genome_path )
chr_files

['chr21.fa.fai',
 'chr3.fa.fai',
 'chr5.fa.fai',
 'chr6.fa.fai',
 'chr8.fa.fai',
 'chr9.fa.fai',
 'chrY.fa.fai',
 '.DS_Store',
 'chr1.fa',
 'chr10.fa',
 'chr11.fa',
 'chr12.fa',
 'chr13.fa',
 'chr14.fa',
 'chr15.fa',
 'chr16.fa',
 'chr17.fa',
 'chr18.fa',
 'chr19.fa',
 'chr2.fa',
 'chr20.fa',
 'chr21.fa',
 'chr22.fa',
 'chr3.fa',
 'chr4.fa',
 'chr5.fa',
 'chr6.fa',
 'chr7.fa',
 'chr8.fa',
 'chr9.fa',
 'chrX.fa',
 'chrY.fa',
 'keep',
 'chr10.fa.fai',
 'chr11.fa.fai',
 'chr12.fa.fai',
 'chr13.fa.fai',
 'chr14.fa.fai',
 'chr15.fa.fai',
 'chr16.fa.fai',
 'chr17.fa.fai',
 'chr18.fa.fai',
 'chr19.fa.fai',
 'chr2.fa.fai',
 'chr20.fa.fai',
 'chr22.fa.fai',
 'chr4.fa.fai',
 'chr7.fa.fai',
 'chrX.fa.fai',
 '._.DS_Store',
 'chr1.fa.fai']

In [4]:
df = read_gtf(gencode_gtf_file_path)

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [5]:
print(df.columns)
print(df.shape)

Index(['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand',
       'frame', 'gene_id', 'gene_type', 'gene_name', 'level', 'hgnc_id',
       'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name',
       'transcript_support_level', 'tag', 'havana_transcript', 'exon_number',
       'exon_id', 'ont', 'protein_id', 'ccdsid'],
      dtype='object')
(3150424, 25)


In [6]:
df.groupby(['feature']).size()

feature
CDS                825028
Selenocysteine        119
UTR                349402
exon              1499012
gene                60649
start_codon         92997
stop_codon          86205
transcript         237012
dtype: int64

In [7]:
promoter_coordinates = [-249, 50]
core_promoter_coordinates= [-34, 35]
donor_coordinates= [-199, 200]
acceptor_coordinates= [-199, 200]

In [15]:
def non_acceptor_sequence(entry, df_acceptor_splice, acceptor_index):
    print ("@@@@", acceptor_index)
    df_acceptor_splice.loc[acceptor_index, ['transcript_id']]= entry['transcript_id']
    df_acceptor_splice.loc[acceptor_index, ['exon_id']] = entry['exon_id']
    df_acceptor_splice.loc[acceptor_index, ['transcript_type']] = entry['transcript_type']
    df_acceptor_splice.loc[acceptor_index, ['strand']]= entry['strand']
    if (entry['strand']=='+'):
        df_acceptor_splice.loc[acceptor_index, ['coordinates']]= int(entry['start'])-1
        acceptor_query= entry['seqname']+':'+str(int(entry['start'])+acceptor_coordinates[0])+'-'+str(int(entry['start'])+acceptor_coordinates[1])
        print ("##$$$", acceptor_query)
        df_acceptor_splice.loc[acceptor_index, ['sequence']] = pybedtools.BedTool.seq(acceptor_query, fasta)
        print ("###", acceptor_index)
    elif (entry['strand']=='-'):
        df_acceptor_splice.loc[acceptor_index, ['coordinates']]= int(entry['end'])+1
        acceptor_query= entry['seqname']+':'+str(int(entry['end'])+(acceptor_coordinates[1]*(-1)))+'-'+str(int(entry['end'])+(acceptor_coordinates[0]*(-1)))
        temp_seq = pybedtools.BedTool.seq(acceptor_query, fasta)
        rev_seq = str(Seq(temp_seq).reverse_complement())
        df_acceptor_splice.loc[acceptor_index, ['sequence']] = rev_seq

In [16]:
def donor_sequence(entry, df_donor_splice, donor_index):
    df_donor_splice.loc[donor_index, ['transcript_id']]= entry['transcript_id']
    df_donor_splice.loc[donor_index, ['exon_id']] = entry['exon_id']
    df_donor_splice.loc[donor_index, ['transcript_type']] = entry['transcript_type']
    df_donor_splice.loc[donor_index, ['strand']]= entry['strand']
    if (entry['strand']=='+'):
        df_donor_splice.loc[donor_index, ['coordinates']] = int(entry['end'])+1
        donor_query= entry['seqname']+':'+str(int(entry['end'])+donor_coordinates[0])+'-'+str(int(entry['end'])+donor_coordinates[1])
        print ("##$$$", donor_query, donor_index)
        df_donor_splice.loc[donor_index, ['sequence']] = pybedtools.BedTool.seq(donor_query, fasta)
    elif (entry['strand']=='-'):
        df_donor_splice.loc[donor_index, ['coordinates']] = int(entry['start'])-1
        donor_query= entry['seqname']+':'+str(int(entry['start'])+(donor_coordinates[1]*(-1)))+'-'+str(int(entry['start'])+(donor_coordinates[0]*(-1)))
        print ("##$$$", donor_query, donor_index)
        temp_seq = pybedtools.BedTool.seq(donor_query, fasta)
        rev_seq = str(Seq(temp_seq).reverse_complement())
        df_donor_splice.loc[donor_index, ['sequence']] = rev_seq

In [17]:
def promoter_sequence(entry, df_promoter, promoter_index):
    df_promoter.loc[promoter_index, ['gene_id']] = entry['gene_id']
    df_promoter.loc[promoter_index, ['transcript_id']]= entry['transcript_id']
    df_promoter.loc[promoter_index, ['transcript_type']]= entry['transcript_type']
    df_promoter.loc[promoter_index, ['strand']] = entry['strand']
    df_promoter.loc[promoter_index, ['exon_id']]=  entry['exon_id']
    if (entry['strand']=='+'):
        df_promoter.loc[promoter_index,['TSS']]= entry['start']
        promoter_query= entry['seqname']+':'+str(int(entry['start'])+promoter_coordinates[0])+'-'+str(int(entry['start'])+promoter_coordinates[1])
        df_promoter.loc[promoter_index, ['sequence']] = pybedtools.BedTool.seq(promoter_query, fasta)
    elif(entry['strand']=='-'):
        df_promoter.loc[promoter_index,['TSS']]= entry['end']
        promoter_query= entry['seqname']+':'+str(int(entry['end'])+(promoter_coordinates[1]*(-1)))+'-'+str(int(entry['end'])+(promoter_coordinates[0]*(-1)))
        temp_seq = pybedtools.BedTool.seq(promoter_query, fasta)
        #print ("Original", temp_seq)
        rev_seq = str(Seq(temp_seq).reverse_complement())
        #print (rev_seq)
        df_promoter.loc[promoter_index, ['sequence']] = rev_seq

In [18]:
def core_promoter_sequence(entry, df_core_promoter, core_promoter_index):
    df_core_promoter.loc[core_promoter_index, ['gene_id']] = entry['gene_id']
    df_core_promoter.loc[core_promoter_index, ['transcript_id']] = entry['transcript_id']
    df_core_promoter.loc[core_promoter_index, ['transcript_type']] = entry['transcript_type']
    df_core_promoter.loc[core_promoter_index, ['strand']] = entry['strand']
    df_core_promoter.loc[core_promoter_index, ['exon_id']]  =  entry['exon_id']
    if (entry['strand']=='+'):
        df_core_promoter.loc[core_promoter_index,['TSS']]= entry['start']
        core_promoter_query= entry['seqname']+':'+str(int(entry['start'])+core_promoter_coordinates[0])+'-'+str(int(entry['start'])+core_promoter_coordinates[1])
        df_core_promoter.loc[core_promoter_index, ['sequence']] = pybedtools.BedTool.seq(core_promoter_query, fasta)
    elif(entry['strand']=='-'):
        df_core_promoter.loc[core_promoter_index,['TSS']]= entry['end']
        core_promoter_query= entry['seqname']+':'+str(int(entry['end'])+(core_promoter_coordinates[1]*(-1)))+'-'+str(int(entry['end'])+(core_promoter_coordinates[0]*(-1)))
        temp_seq = pybedtools.BedTool.seq(core_promoter_query, fasta)
        rev_seq = str(Seq(temp_seq).reverse_complement())
        df_core_promoter.loc[core_promoter_index, ['sequence']] = rev_seq

In [ ]:
global df_promoter, df_core_promoter, df_donor_splice, df_acceptor_splice


for file in chr_files:
    if file.endswith('.fa'):
        reference_genome_file_path = reference_genome_path + file
        chr_name = file.split('.')[0]
        print (chr_name)
        df_chr= df[df['seqname']==chr_name]
        df_new = df_chr[df_chr['feature']=='exon'].reset_index(drop=True)
        print (df_new.shape)
        #print (reference_genome_file_path)
        fasta = pybedtools.example_filename(reference_genome_file_path)
        
        
        folder_name = data_save_path + chr_name   
        print (folder_name)
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
        
        
        #df_promoter = pd.DataFrame(columns= ['gene_id','transcript_id','strand','exon_id','transcript_type','TSS', 'sequence'])
        #df_core_promoter = pd.DataFrame(columns= ['gene_id','transcript_id','strand','exon_id','transcript_type','TSS','sequence'])
        df_donor_splice = pd.DataFrame(columns= ['transcript_id','exon_id','transcript_type','strand','coordinates','sequence'])
        df_acceptor_splice= pd.DataFrame(columns= ['transcript_id','exon_id', 'transcript_type','strand','coordinates', 'sequence'])
        promoter_index = core_promoter_index = donor_index = acceptor_index = 0
        
        
        
        
        df_by_transcript = df_new.head(20).groupby("transcript_id")
        for transcript, entries in df_by_transcript:
            print ("\n", transcript,len( entries) )
            print (entries.iloc[0]['strand'])
            if (entries.iloc[0]['strand']=='+'):
                for index, entry in entries.iterrows():
                    if (int(entry['exon_number'])==1):
                        print("TSS", entry['start'])
                        #promoter_sequence(entry, df_promoter, promoter_index)
                        #promoter_index= promoter_index+1
                        #core_promoter_sequence(entry, df_core_promoter, core_promoter_index)
                        #core_promoter_index = core_promoter_index +1
                        donor_sequence(entry, df_donor_splice, donor_index)
                        donor_index =donor_index+ 1
                    elif (int(entry['exon_number'])==len(entries)):
                        print ("Last acceptor_splice_site",  entry['start'])
                        acceptor_sequence(entry, df_acceptor_splice, acceptor_index)
                        acceptor_index = acceptor_index +1
                    else:
                        print ("Acceptor splice site", entry['start'])
                        acceptor_sequence(entry, df_acceptor_splice, acceptor_index)
                        acceptor_index = acceptor_index +1
                        print ("Donor splice site", entry['end'])
                        donor_sequence(entry, df_donor_splice, donor_index)
                        donor_index =donor_index+ 1
            elif (entries.iloc[0]['strand']=='-'):
                for index, entry in entries.iterrows():
                    if (int(entry['exon_number'])==1):
                        print("TSS", entry['end'])
                        print("first donor_splice_site", entry['start'])
                        #promoter_sequence(entry, df_promoter, promoter_index)
                        #promoter_index= promoter_index+1
                        #core_promoter_sequence(entry, df_core_promoter, core_promoter_index)
                        #core_promoter_index = core_promoter_index +1
                        donor_sequence(entry, df_donor_splice, donor_index)
                        donor_index =donor_index+ 1
                    elif (int(entry['exon_number'])==len(entries)):
                        print ("Last acceptor_splice_site",  entry['end'])
                        acceptor_sequence(entry, df_acceptor_splice, acceptor_index)
                        acceptor_index = acceptor_index +1
                    else:
                        print ("Acceptor splice site", entry['end'])
                        acceptor_sequence(entry, df_acceptor_splice, acceptor_index)
                        acceptor_index = acceptor_index +1
                        print ("Donor splice site", entry['start'])
                        donor_sequence(entry, df_donor_splice, donor_index)
                        donor_index =donor_index+ 1
        df_promoter.to_csv(folder_name +"/"+ entry['seqname']+"_promoter.csv", index=False)
        df_core_promoter.to_csv(folder_name +"/"+ entry['seqname']+"_core_promoter_sequence.csv", index=False)
        df_acceptor_splice.to_csv(folder_name +"/"+ entry['seqname']+"_acceptor_splice_set_sequence.csv", index=False)
        df_donor_splice.to_csv(folder_name +"/"+ entry['seqname']+"_donor_splice_set.csv", index=False)
        break()

In [48]:
df_promoter

,gene_id,transcript_id,strand,exon_id,transcript_type,TSS,sequence
0,ENSG00000237613.2,ENST00000417324.1,-,ENSE00001656588.1,lncRNA,36081,AGAATTGGTTTGATTTACATAATCCTGCCCCTGAGATTTAGCCCCA...
1,ENSG00000223972.5,ENST00000450305.2,+,ENSE00001948541.1,transcribed_unprocessed_pseudogene,12010,gccggtgttgagaatgactgCGCAAATTTGCCGGATTTCCTTTGCT...
2,ENSG00000223972.5,ENST00000456328.2,+,ENSE00002234944.1,processed_transcript,11869,atgcctgtttctccacaaagtgtttacttttggatttttgccagtc...
3,ENSG00000237613.2,ENST00000461467.1,-,ENSE00001618781.2,lncRNA,36073,TTTGATTTACATAATCCTGCCCCTGAGATTTAGCCCCACCCCTGAG...
4,ENSG00000238009.6,ENST00000466430.5,-,ENSE00001606755.2,lncRNA,120932,TCTTTACTTGCACTGACAAATATGAACGCTGGGGAATTTGGAAatg...
5,ENSG00000243485.5,ENST00000469289.1,+,ENSE00001841699.1,lncRNA,30267,ttgggcagctcgtgttcaatgggtagagtttcaggctggggtgatg...
6,ENSG00000238009.6,ENST00000471248.1,-,ENSE00001934975.1,lncRNA,129173,gactccatcttataaaaggaaaaaagaaagaaaagaaaaaTTCCAT...
7,ENSG00000243485.5,ENST00000473358.1,+,ENSE00001947070.1,lncRNA,29554,CCGGACGCCTCCTCACCTGCGAGCCGCCCTCCCGGAAGCTCCCGCC...
8,ENSG00000238009.6,ENST00000477740.5,-,ENSE00001919246.1,lncRNA,129217,agctgagatcgcaccagtgcactccagcctgggcaacagagtgaga...
9,ENSG00000227232.5,ENST00000488147.1,-,ENSE00001890219.1,unprocessed_pseudogene,29570,GCTGAGAGACGCAGGAAAGCGGGTCAAggcgtagggctggagggca...


In [49]:
df_core_promoter

,gene_id,transcript_id,strand,exon_id,transcript_type,TSS,sequence
0,ENSG00000237613.2,ENST00000417324.1,-,ENSE00001656588.1,lncRNA,36081,CAAGGATTAGTCCTGCTGCCCTGTGCCCAGCTCCCACACAACGGGG...
1,ENSG00000223972.5,ENST00000450305.2,+,ENSE00001948541.1,transcribed_unprocessed_pseudogene,12010,CACAGGGTCTTGATGCTGTGGTCTTCATCTGCAGGTGTCTGACTTC...
2,ENSG00000223972.5,ENST00000456328.2,+,ENSE00002234944.1,processed_transcript,11869,CCAGCACCGGGTATCATTCACCATTTTTCTTTTCGTTAACTTGCCG...
3,ENSG00000237613.2,ENST00000461467.1,-,ENSE00001618781.2,lncRNA,36073,AGTCCTGCTGCCCTGTGCCCAGCTCCCACACAACGGGGTTTCGGGG...
4,ENSG00000238009.6,ENST00000466430.5,-,ENSE00001606755.2,lncRNA,120932,TTTCCATGTGATTTTCCTCTTAATTTTTTTCTAGCTGATCCATATG...
5,ENSG00000243485.5,ENST00000469289.1,+,ENSE00001841699.1,lncRNA,30267,GCTTTTTTAAAAAGTTAAATATATAAAAAGAGCATCATCAGTCCAA...
6,ENSG00000238009.6,ENST00000471248.1,-,ENSE00001934975.1,lncRNA,129173,AGCATCTTAAAATGATTCAACAGGAGGAGATAAGGAAGCTCGAGGA...
7,ENSG00000243485.5,ENST00000473358.1,+,ENSE00001947070.1,lncRNA,29554,GGCACGGAACTCACGCTCACTCCGAGCTCCCGACGTGCACACGGCT...
8,ENSG00000238009.6,ENST00000477740.5,-,ENSE00001919246.1,lncRNA,129217,GTGTCACAACTGCACATTGGTTTCACAGCTGCAGGACAAGTTCGAG...
9,ENSG00000227232.5,ENST00000488147.1,-,ENSE00001890219.1,unprocessed_pseudogene,29570,GGGTAGGGGCGGCCTGACGCTCGGAAGACAACGCATGGGAGCCGTG...


In [50]:
df_donor_splice

,transcript_id,exon_id,transcript_type,strand,coordinates,sequence
0,ENST00000417324.1,ENSE00001656588.1,lncRNA,-,35720,GGT
1,ENST00000417324.1,ENSE00001669267.1,lncRNA,-,35276,tgt
2,ENST00000450305.2,ENSE00001948541.1,transcribed_unprocessed_pseudogene,+,12058,GCA
3,ENST00000450305.2,ENSE00001671638.2,transcribed_unprocessed_pseudogene,+,12228,AGT
4,ENST00000450305.2,ENSE00001758273.2,transcribed_unprocessed_pseudogene,+,12698,TGT
5,ENST00000450305.2,ENSE00001799933.2,transcribed_unprocessed_pseudogene,+,13053,GGC
6,ENST00000450305.2,ENSE00001746346.2,transcribed_unprocessed_pseudogene,+,13375,AGT
7,ENST00000456328.2,ENSE00002234944.1,processed_transcript,+,12228,AGT
8,ENST00000456328.2,ENSE00003582793.1,processed_transcript,+,12722,GGT
9,ENST00000461467.1,ENSE00001618781.2,lncRNA,-,35720,GGT


In [46]:
df_acceptor_splice

,transcript_id,exon_id,transcript_type,strand,coordinates,sequence
0,ENST00000450305.2,ENSE00001671638.2,transcribed_unprocessed_pseudogene,+,12178,GAT
1,ENST00000450305.2,ENSE00001758273.2,transcribed_unprocessed_pseudogene,+,12612,AGG
2,ENST00000450305.2,ENSE00001799933.2,transcribed_unprocessed_pseudogene,+,12974,AGC
3,ENST00000450305.2,ENSE00001746346.2,transcribed_unprocessed_pseudogene,+,13220,AGG
4,ENST00000450305.2,ENSE00001863096.1,transcribed_unprocessed_pseudogene,+,13452,AGT
5,ENST00000456328.2,ENSE00003582793.1,processed_transcript,+,12612,AGG
6,ENST00000456328.2,ENSE00002312635.1,processed_transcript,+,13220,AGG
7,ENST00000473358.1,ENSE00001922571.1,lncRNA,+,30563,AGT
8,ENST00000473358.1,ENSE00001827679.1,lncRNA,+,30975,agt
9,ENST00000488147.1,ENSE00003507205.1,unprocessed_pseudogene,-,24892,AGG


In [41]:
# df_promoter.to_csv(data_save_path +"/"+ entry['seqname']+"_promoter.csv", index=False)
# df_core_promoter.to_csv(data_save_path +"/"+ entry['seqname']+"_core_promoter_sequence.csv", index=False)
# df_acceptor_splice.to_csv(data_save_path +"/"+ entry['seqname']+"_acceptor_splice_set_sequence.csv", index=False)
# df_donor_splice.to_csv(data_save_path +"/"+ entry['seqname']+"_donor_splice_set.csv", index=False)

In [46]:
test_seq = pybedtools.BedTool.seq('chr1:11628-11927', fasta)
test_seq

'ttctccacaaagtgtttacttttggatttttgccagtctaacaggtgaAGccctggagattcttattagtgatttgggctggggcctggccatgtgtatttttttaaatttccactgatgattttgctgcatggccggtgttgagaatgactgCGCAAATTTGCCGGATTTCCTTTGCTGTTCCTGCATGTAGTTTAAACGAGATTGCCAGCACCGGGTATCATTCACCATTTTTCTTTTCGTTAACTTGCCGTCAGCCTTTTCTTTGACCTCTTCTTTCTGTTCATGTGTATTTGCTGT'

In [47]:
seq = Seq(test_seq)

print (seq.reverse_complement())

ACAGCAAATACACATGAACAGAAAGAAGAGGTCAAAGAAAAGGCTGACGGCAAGTTAACGAAAAGAAAAATGGTGAATGATACCCGGTGCTGGCAATCTCGTTTAAACTACATGCAGGAACAGCAAAGGAAATCCGGCAAATTTGCGcagtcattctcaacaccggccatgcagcaaaatcatcagtggaaatttaaaaaaatacacatggccaggccccagcccaaatcactaataagaatctccagggCTtcacctgttagactggcaaaaatccaaaagtaaacactttgtggagaa


In [14]:
reference_genome_file_path = '/mnt/data05/shared/pdutta_data/Human_Genome_Data/UCSC/genome_by_sequence/chr1.fa'
fasta = pybedtools.example_filename(reference_genome_file_path)

In [33]:
temp_seq = pybedtools.BedTool.seq('chr1:1-248956422', fasta)

In [34]:
len(temp_seq)

248956422

In [68]:
import sys
sys.getsizeof(temp_seq,'U6') 

248956471

In [70]:
np_test_seq  = np.array(list(temp_seq))

In [72]:
sys.getsizeof(np_test_seq,'U6') 

995825792

In [115]:
test_str= "HGGVHGJBKJGNBJHBNBHJBNHGJHBHB"

In [69]:
import numpy as np
test_str_new = np.array(test_str)


In [61]:
test_str_new

array('HGGVHGJBKJGNBJHBNBHJBNHGJHBHB', dtype='<U29')

In [36]:
position = [[10,12], [16,20]]

In [119]:
for i in position:
    print (i)
    sub_string = test_str[i[0]:i[1]+1]
    print(sub_string)
    substitute_str = "$"* int(i[1]+1-i[0])
    print (substitute_str)
    test_str = test_str.replace(sub_string, substitute_str)
    print (test_str)
line = re.sub('[$]', '', test_str)
print (line)

[10, 12]
$$$
$$$
HGGVHGJBKJ$$$JHB$$$$$NHGJHBHB
[16, 20]
$$$$$
$$$$$
HGGVHGJBKJ$$$JHB$$$$$NHGJHBHB
HGGVHGJBKJJHBNHGJHBHB


In [51]:
# test_str_new[position[0][0]:position[0][1]] = 'NN'
test_str_new[0][0]

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [12]:
test_str = test_str[0:position[0]]+ test_str[position[1]:]

In [94]:
test_str = test_str.replace(test_str[0], '$')
test_str

'$GGV$GJBKJGNBJ$BNB$JBN$GJ$B$B'

In [ ]:
for 

In [54]:
check_sr= ""
for i in range(10):
    check_sr= check_sr+ '0'

In [55]:
check_sr

'0000000000'

In [56]:
check_sr1= ""
check_sr1 = ["0" for i in range(10)  ]

In [57]:
check_sr1

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']

In [58]:
checksr2= ''.join(["0" for i in range(10)])

In [59]:
checksr2

'0000000000'

In [63]:
A = np.array([0,1,2,3,4,5,6])

In [64]:
A

array([0, 1, 2, 3, 4, 5, 6])

In [66]:
A[0:2] = "N"

ValueError: invalid literal for int() with base 10: 'N'

In [77]:
for  i in range([10:12]):
    print(i)

SyntaxError: invalid syntax (2966232614.py, line 1)

In [78]:
str1 = "0"*5

In [79]:
str1

'00000'

In [84]:
p = range(1,4)

In [85]:
p

range(1, 4)

In [86]:
p[0]

1

In [98]:
a_string = "Welme to Kite! CO"

index = a_string.find("co")

print(index)

-1


In [100]:
import re

string = 'i think that that that that student wrote there is not that right'
pattern = 'that'

print([match.start() for match in re.finditer(pattern, string)])

[8, 13, 18, 23, 55]


In [104]:
pattern1= 'GT'

In [106]:
Non_GT_donor = [match.start() for match in re.finditer(pattern1, temp_seq)]
len(Non_GT_donor)

5479664

In [112]:
Non_GT_donor[0:10]

[11453, 11492, 11497, 11806, 11816, 11819, 11844, 11868, 11879, 11908]

TypeError: 'NoneType' object is not subscriptable